In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [ ]:
dataset = pd.read_csv('covtype.data', names=list(range(55)))
dataset.head(5)

In [ ]:
# Split dataset for train- test                  0
y_train = dataset.iloc[:15120,(dataset.shape[1]-1)]
y_test = dataset.iloc[15120:,(dataset.shape[1]-1)]

X_train=dataset.iloc[:15120,:(dataset.shape[1]-1)]
X_test= dataset.iloc[15120:,:(dataset.shape[1]-1)]

scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test= scaler.transform(X_test)


In [ ]:
# Grid search for tuning svm
Cs = [2**(-3),2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10),2**(11),2**(12),2**(13),2**(14),2**(15)]
gammas = [2**(-15),2**(-14),2**(-13),2**(-12),2**(-11),2**(-10),2**(-9),2**(-8),2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]

parameters = {'C':Cs,'gamma':gammas}
svc = svm.SVC( kernel='rbf',decision_function_shape='ovo')
#cv = StratifiedKFold(y=y_train, n_folds=10) kai katw cv=cv
clf = GridSearchCV(svc, parameters, cv=10,scoring='accuracy',verbose=2,n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
# save results to text
with open('out.txt2', 'w') as f:
    print(clf.cv_results_, file=f)
    
with open('out_values_per_pair2.txt', 'w') as f:
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=f)
        #print(clf.cv_results_, file=f)


In [ ]:
print(clf.best_params_)
print("The best parameters are %s with a score of %f" % (clf.best_params_, clf.best_score_))

In [ ]:
# predict test set with chosen parameters
svc2 = svm.SVC( kernel='rbf',C=16,gamma=8)
svc2.fit(X_train, y_train)
y_hat = svc2.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_hat)
print("SVM accuracy: %f" % (acc))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_hat))
print(svc2.support_vectors_ )
print("num of vectors per class",svc2.n_support_)
print("sum of vectors: ",np.sum(svc2.n_support_))

In [ ]:
print(svc2.support_vectors_.shape)